# Creating and running a fund
This script will import the vrious models and run them as funds from 01-01-2014 to find the best performing fund/model.

In [1]:
#Import models
import numpy as np
import pandas as pd
import math
import os
import tables
from rf_modules import *

In [2]:
#Setup variables
path = r'C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices'

In [3]:
#Create a class for maintaining the value of the fund
class new_fund:
    def __init__(self,init_val):
        self.st_val = init_val #The starting value of the fund
        self.available = init_val #The amount available to invest, initially set to the investment value
        self.invested_value = 0 #The total value of hel stocks
        self.codb = 0 #The sum cost of doing business (stamp duty, trading fees)
        self.ledger = [] #A running total of the trades made in this fund
        self.cur_holdings = {} #A dictionary of the current fund holdings
        print('NEW FUND')
        print('st_val:{:,}'.format(self.st_val))
        print('available:{:,}'.format(self.available))
    @property
    def fund_value(self):
        return self.invested_value + self.available
    #Create a function to buy shares
    def buy(self
            ,ticker:str
            ,trade_date:int
            ,price:float
            ,spread:float
            ,value:float
            ,trade_cost:float
            ,signal_prob:float
            ,sd_rate:float = 0.005
            ,val_inc_tc:bool = True
            ,_verbose:bool = True):
        if _verbose == True:
            print('\nBUY {}'.format(ticker))
        #Check if already bought
        if ticker in self.cur_holdings:
            if _verbose == True:
                print('\t{} is already in the fund, ignore and move to the next row'.format(ticker))
            return
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {:,.2f}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {:,.2f}'.format(price))
        #Calculate the trade value
        if val_inc_tc == True: #Does the value include the trade cost
            trade_funds = value - trade_cost
        else:
            trade_funds = value
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the number of whole shares which can be purchased
        share_vol = int((trade_funds / (1+sd_rate))/a_price)
        trade_value = round(share_vol * a_price,2)
        stamp_duty = round(trade_value * sd_rate,2)
        #Calc the ledger_value
        spread_cost = round(share_vol * price * spread,2)
        ledger_value = -round(trade_value + trade_cost + stamp_duty,2)
        #Check the fund has the money to cover this trade
        if -ledger_value > self.available:
            print('\tnot enough funds')
            raise ValueError('you do not have the funds to make this trade -> this transaction will be cancelled')
        #Create a record for the ledger
        ledge_rec = {
            'trade_type':'buy'
            ,'signal_prob':signal_prob
            ,'ticker':ticker
            ,'trade_date':trade_date
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'stamp_duty':stamp_duty
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'ledger_value':ledger_value
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value + (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available + ledger_value
        }
        self.ledger.append(ledge_rec)
        if _verbose == True:
            print('\tLEDGER ENTRY -> {}'.format(ledge_rec)) #OPTIONAL - PRINT ENTRY
        #Update the object
        self.available += round(ledger_value,2)
        self.codb += round(trade_cost + spread_cost + stamp_duty,2)
        self.invested_value += round(share_vol * price,2)
        #Add to cur_holdings
        holding_rec = {
           'share_vol':share_vol
            ,'cur_price':price
            ,'value':round(share_vol * price,2)
        }
        #Check if key already in dict
        if ticker in self.cur_holdings: #Should not happen due to earlier exit if key in cur_holdings
            self.cur_holdings[ticker]['share_vol'] += share_vol
            self.cur_holdings[ticker]['cur_price'] = round(price,2)
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
        else:
            self.cur_holdings[ticker] = holding_rec
        
    #Create a function to sell shares
    def sell(self
             ,ticker:str
             ,trade_date:int
             ,price:float
             ,spread:float
             ,trade_cost:float
             ,signal_prob:float
             ,sd_rate:float = 0.00
            ,_verbose:bool = True):
        if _verbose == True:
            print('\nSELL {}'.format(ticker))
        #Check if there are holdings to be sold
        if ticker not in self.cur_holdings:
            if _verbose == True:
                print('\t{} is not in the fund, ignore and move to the next row'.format(ticker))
            return
        #Round down the share_vol
        share_vol = int(self.cur_holdings[ticker]['share_vol'])
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {}'.format(price))
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the trade value
        trade_value = round(share_vol*b_price,2)
        stamp_duty = round(trade_value * sd_rate,2)
        value = round(trade_value - trade_cost - stamp_duty,2)
        spread_cost = round(share_vol * price * spread)
        #Calc the ledger_value
        ledger_value = round(trade_value - trade_cost - stamp_duty,2)
        #Create a record for the ledger
        ledge_rec = {
           'trade_type':'sell'
            ,'signal_prob':signal_prob
            ,'ticker':ticker
            ,'trade_date':trade_date
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'stamp_duty':stamp_duty
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'ledger_value':ledger_value
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value - (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available + ledger_value
        }
        self.ledger.append(ledge_rec)
        if _verbose == True:
            print('\tLEDGER ENTRY -> {}'.format(ledge_rec)) #OPTIONAL - PRINT ENTRY
        #Update the object
        self.available += round(ledger_value,2)
        self.codb += round(trade_cost + spread_cost + stamp_duty,2)
        self.invested_value += -round((share_vol*price),2)
        #Remove from cur_holdings
        #Check if key already in dict
        if ticker in self.cur_holdings:
            if self.cur_holdings[ticker]['share_vol'] > share_vol:
                self.cur_holdings[ticker]['share_vol'] += -share_vol
                self.cur_holdings[ticker]['cur_price'] = price
                self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
            elif self.cur_holdings[ticker]['share_vol'] == share_vol:
                del self.cur_holdings[ticker] #Delete from the dictionary
            else:
                raise ValueError('you do not have enough shares to make this trade. You want to sell {} of {} however you only have {}'.format(share_vol,ticker,self.cur_holdings[ticker]['share_vol']))
        else:
            return
        
    #Create a function to update value after a price change
    def price_change(self,ticker:str,price:float):
        #Check if key already in dict
        if ticker in self.cur_holdings:
            self.invested_value += round((self.cur_holdings[ticker]['share_vol']*price) - self.cur_holdings[ticker]['value'],2)
            self.cur_holdings[ticker]['cur_price'] = price
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
        else:
            return

In [4]:
def completed_trades(_df_in,_init_val:int=1,_cur_holdings:dict={}):
    #From the ledger create a dataframe of completed trades
    _completed_trades = {}
    #Format
    #     ABC:{
    #         open_position:True/False #Bool showing if there is currently an open position
    #         ,trades:[ #List showing all trades
    #             { #Each trade has an object
    #                 share_vol:12345 #Volume of shares purchased
    #                 ,buy_spend:12345.67 #Total value spent in buying shares including costs
    #                 ,sell_spend:12345.67 #Total value spent in selling shares including costs
    #                 ,profit_loss:12345.67 #Profit/loss of ths trade
    #                 ,holding_value:12345.67 #Current value of the shares
    #             }
    #         ]
    #     }
    for _index,_row in _df_in.iterrows():
        #Check if there is an open trade for this ticker
        if _row['ticker'] not in _completed_trades:
            _completed_trades[_row['ticker']] = {
                'open_position':False
                ,'trades':[]
            }
        #Deal with buying
        if _row['trade_type'] == 'buy':
            #Create a trade object and add to the trades list in completed_trades
            _completed_trades[_row['ticker']]['trades'].append({
                'ticker':_row['ticker']
                ,'share_vol':_row['share_vol']
                ,'buy_price':_row['price']
                ,'buy_spend':-_row['ledger_value']
                ,'buy_prob':_row['signal_prob']
                ,'buy_date':_row['trade_date']
                ,'sell_price':None
                ,'sell_spend':None
                ,'sell_prob':None
                ,'sell_date':None
                ,'periods_held':None
                ,'profit_loss':None
                ,'holding_value':_row['holding_value']
            })
            #open the trading position
            _completed_trades[_row['ticker']]['open_position'] = True
        #Dealing with selling
        elif _row['trade_type'] == 'sell':
            _shares_to_sell = _row['share_vol']
            #Find open positions and sell until shares al gone
            for _trade in _completed_trades[_row['ticker']]['trades']:
                if _trade['share_vol'] == _row['share_vol']:
                    _trade['sell_price'] = _row['price']
                    _trade['sell_spend'] = _row['ledger_value']
                    _trade['sell_prob'] = _row['signal_prob']
                    _trade['sell_date'] = _row['trade_date']
                    _trade['holding_value'] = _row['holding_value']
    _trades_li = []
    for _tick in _completed_trades:
        for _trade in _completed_trades[_tick]['trades']:
            _trades_li.append(_trade)
    _trades_df = pd.DataFrame(_trades_li,columns=[
                'ticker'
                ,'share_vol'
                ,'buy_price'
                ,'buy_spend'
                ,'buy_prob'
                ,'buy_date'
                ,'sell_price'
                ,'sell_spend'
                ,'sell_prob'
                ,'sell_date'
                ,'periods_held'
                ,'profit_loss'
                ,'holding_value'])
    _trades_df['profit_loss'] = _trades_df['sell_spend'] - _trades_df['buy_spend'] 
    _trades_df['periods_held'] = _trades_df['sell_date'] - _trades_df['buy_date']
    _trades_df['periods_held'] = [_x.days/7 for _x in _trades_df['periods_held']]
    #Dealing with open trade values
    _trades_df.reset_index(inplace=True,drop=True)
    for _index,_row in _trades_df[_trades_df.sell_date.isna()].iterrows():
        _trades_df.iloc[_index,_trades_df.columns.get_loc('holding_value')] = _cur_holdings[_row.ticker]['value']
    #Display top level summary
    print('TRADE COUNT -> {:,}'.format(_trades_df[~_trades_df['profit_loss'].isna()].shape[0]))
    print('TOTAL PROFIT/LOSS -> £{:,.2f}'.format(_trades_df['profit_loss'].sum()/100))
    print('\nOPEN TRADES -> {:,}'.format(_trades_df[_trades_df['profit_loss'].isna()].shape[0]))
    print('OPEN TRADES VALUE -> £{:,.2f}'.format(_trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum()/100))
    print('\nTOTAL ROI VALUE -> £{:,.2f}'.format((_trades_df['profit_loss'].sum()/100) + (_trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum()/100)))
    print('TOTAL ROI % -> {:,.2f}%'.format(100 * (_trades_df['profit_loss'].sum() + _trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum() - _init_val) / _init_val))
    #Calculate ROI
    _trades_df['roi'] = (_trades_df['holding_value'] / _trades_df['buy_spend']) - 1
    print('''\nROI STATS:
        AVERAGE ROI % -> {:.2f}
        AVERAGE POS ROI % -> {:.2f}
        POS COUNT -> {:,.0f}
        AVERAGE NEG ROI % -> {:.2f}
        NEG COUNT -> {:,.0f}
    \nPERIODS HELD STATS:
        AVERAGE PERIODS HELD -> {:.2f}
        AVERAGE POS ROI PERIODS HELD -> {:.2f}
        AVERAGE NEG ROI PERIODS HELD -> {:.2f}
        '''.format(
        _trades_df['roi'].mean()*100
        ,_trades_df[_trades_df['roi'] > 0]['roi'].mean()*100
        ,len(_trades_df[_trades_df['roi'] > 0])
        ,_trades_df[_trades_df['roi'] <= 0]['roi'].mean()*100
        ,len(_trades_df[_trades_df['roi'] <= 0])
        ,_trades_df['periods_held'].mean()
        ,_trades_df[_trades_df['roi'] > 0]['periods_held'].mean()
        ,_trades_df[_trades_df['roi'] <= 0]['periods_held'].mean()
    ))
    
    return _trades_df

In [5]:
def run_fund(_df_in,_fund_value_st:float=1000000,_investment_limit_min_val:float=100000,_investment_limit_max_per:float=0.1,_spread:float=0.01,_trade_cost:float=250):
    #Run through rows and buy and sell according to signals and holdings
    _run_time = process_time()
    _fund = new_fund(_fund_value_st)
    #Error object
    _errors_li = []
    for _index,_row in _df_in.iterrows():
        try:
            #Follow signal
            if _row['signal'] =='buy':
                #Check for funds
                if _fund.available < _investment_limit_min_val:
                    continue
                #Buy shares
                _val_to_invest = _fund.available*_investment_limit_max_per if _fund.available*_investment_limit_max_per > _investment_limit_min_val else _investment_limit_min_val
                _fund.buy(_row['ticker'],_row['date'],_row['open_shift_neg1'],_spread,_val_to_invest,_trade_cost,_row['signal_prob'])
            elif _row['signal'] =='sell':
                #Sell shares
                _fund.sell(_row['ticker'],_row['date'],_row['open_shift_neg1'],_spread,_trade_cost,_row['signal_prob'])
            elif _row['signal'] == 'hold':
                _fund.price_change(_row['ticker'],_row['close'])
        except Exception as e:
            print('ERROR: {}'.format(e))
            _errors_li.append({
                'error':type(e)
                ,'error_desc':e
            })
    _run_time.end()
    #Check for errors
    print('\n\nERROR COUNT -> {}'.format(len(_errors_li)))
    if len(_errors_li) > 0:
        print('\tSHOWING ERRORS')
        for e in _errors_li:
            print('\t{}'.format(e))
    return _fund

In [10]:
#Establish trading variables
#All figures in pence
fund_vars = {
    '_fund_value_st':1000000 #£10,000
    ,'_trade_cost':250 #£2.50
    ,'_investment_limit_min_val':100000 #£1,000
    ,'_investment_limit_max_per':0.1 #10%
    ,'_spread':0.01 #1%
}

# Light Gradient Boost
Steps:
1. Import the output from running the model
2. Run the model through the fund

In [11]:
#Import and combine prices files
df_signals_lgb = pd.read_hdf(path +r'\historic_lgb_bsh_signals.h5')
print('SHAPE: {}'.format(df_signals_lgb.shape))
print(df_signals_lgb.columns)
df_signals_lgb.head()

SHAPE: (94306, 16)
Index(['ticker', 'date', 'high', 'low', 'volume', 'open', 'close', 'change',
       'ema12', 'ema26', 'macd_line', 'signal_line', 'macd', 'open_shift_neg1',
       'signal', 'signal_prob'],
      dtype='object')


,ticker,date,high,low,volume,open,close,change,ema12,ema26,macd_line,signal_line,macd,open_shift_neg1,signal,signal_prob
314,3IN,2014-01-06,190.56,187.61,2305949.0,189.72,189.72,0.00,187.437790,187.351381,0.086408,-0.035103,0.121512,188.87,hold,0.848095
315,3IN,2014-01-13,190.56,188.30,2091342.0,188.87,189.58,0.71,187.767361,187.516464,0.250896,0.022097,0.228800,188.87,hold,0.794872
316,3IN,2014-01-20,190.00,187.47,2386674.0,188.87,189.58,0.71,188.046228,187.669319,0.376910,0.093059,0.283850,188.17,hold,0.825625
317,3IN,2014-01-27,190.84,188.17,1783577.0,188.17,190.84,2.67,188.476039,187.904184,0.571855,0.188819,0.383037,189.44,hold,0.792064
318,3IN,2014-02-03,190.42,188.31,1742104.0,189.44,189.72,0.28,188.667418,188.038689,0.628729,0.276801,0.351928,188.59,hold,0.842516


In [12]:
#Order the data by date (asc) and buy probability (desc)
signal_df = df_signals_lgb[['ticker','date','open','high','low','close','open_shift_neg1','signal','signal_prob']].copy()
signal_df.sort_values(['date','signal_prob'],ascending=[True,False],inplace=True)
signal_df.reset_index(drop=True,inplace=True)
print(signal_df.signal.value_counts())
signal_df.head()

hold    85340
sell     5895
buy      3071
Name: signal, dtype: int64


,ticker,date,open,high,low,close,open_shift_neg1,signal,signal_prob
0,3IN,2014-01-06,189.72,190.56,187.61,189.72,188.87,hold,0.848095
1,FOUR,2014-01-06,650.50,668.28,650.50,665.50,655.50,hold,0.808841
2,BGFD,2014-01-06,369.00,387.71,368.44,384.00,385.00,hold,0.789082
3,BBY,2014-01-06,289.90,295.88,286.96,289.80,289.70,hold,0.765045
4,FRES,2014-01-06,772.00,781.00,658.00,679.00,690.00,hold,0.763553


In [13]:
#Run through rows and buy and sell according to signals and holdings
lgb_fund = run_fund(signal_df,**fund_vars)

NEW FUND
st_val:1,000,000
available:1,000,000

BUY AGR
	LEDGER ENTRY -> {'trade_type': 'buy', 'signal_prob': 0.6350299794409866, 'ticker': 'AGR', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 40.0, 'ask_price': 40.4, 'bid_price': 39.6, 'share_vol': 2456, 'trade_value': 99222.4, 'stamp_duty': 496.11, 'trade_cost': 250, 'spread_cost': 982.4, 'holding_value': 98240.0, 'ledger_value': -99968.51, 'invested_pre_trade': 0, 'invested_post_trade': 98240.0, 'available_pre_trade': 1000000, 'available_post_trade': 900031.49}

SELL GVC
	GVC is not in the fund, ignore and move to the next row

BUY DC
	LEDGER ENTRY -> {'trade_type': 'buy', 'signal_prob': 0.5783653911013563, 'ticker': 'DC', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 277.0, 'ask_price': 279.77, 'bid_price': 274.23, 'share_vol': 354, 'trade_value': 99038.58, 'stamp_duty': 495.19, 'trade_cost': 250, 'spread_cost': 980.58, 'holding_value': 98058.0, 'ledger_value': -99783.77, 'invest


SELL BNZL
	BNZL is not in the fund, ignore and move to the next row

SELL WPP
	LEDGER ENTRY -> {'trade_type': 'sell', 'signal_prob': 0.4044802399188826, 'ticker': 'WPP', 'trade_date': Timestamp('2014-04-14 00:00:00'), 'spread': 0.01, 'price': 1269.0, 'ask_price': 1281.69, 'bid_price': 1256.31, 'share_vol': 78, 'trade_value': 97992.18, 'stamp_duty': 0.0, 'trade_cost': 250, 'spread_cost': 990, 'holding_value': 98982.0, 'ledger_value': 97742.18, 'invested_pre_trade': 999965.75, 'invested_post_trade': 900983.75, 'available_pre_trade': 3153.0599999999104, 'available_post_trade': 100895.2399999999}

BUY GFTU
	LEDGER ENTRY -> {'trade_type': 'buy', 'signal_prob': 0.5612805620396483, 'ticker': 'GFTU', 'trade_date': Timestamp('2014-04-21 00:00:00'), 'spread': 0.01, 'price': 610.5, 'ask_price': 616.61, 'bid_price': 604.39, 'share_vol': 160, 'trade_value': 98657.6, 'stamp_duty': 493.29, 'trade_cost': 250, 'spread_cost': 976.8, 'holding_value': 97680.0, 'ledger_value': -99400.89, 'invested_pre_tra

SELL PFG
	PFG is not in the fund, ignore and move to the next row

SELL DLG
	DLG is not in the fund, ignore and move to the next row

SELL JAM
	JAM is not in the fund, ignore and move to the next row

SELL MYI
	MYI is not in the fund, ignore and move to the next row

SELL GCP
	GCP is not in the fund, ignore and move to the next row

SELL BBGI
	BBGI is not in the fund, ignore and move to the next row

SELL INPP
	INPP is not in the fund, ignore and move to the next row

SELL AVV
	AVV is not in the fund, ignore and move to the next row

SELL VOF
	VOF is not in the fund, ignore and move to the next row

SELL IMB
	IMB is not in the fund, ignore and move to the next row

SELL PFG
	PFG is not in the fund, ignore and move to the next row

SELL INPP
	INPP is not in the fund, ignore and move to the next row

SELL RDSB
	RDSB is not in the fund, ignore and move to the next row

SELL RSE
	RSE is not in the fund, ignore and move to the next row

SELL BLND
	BLND is not in the fund, ignore and move to


SELL FOUR
	FOUR is not in the fund, ignore and move to the next row

SELL CBG
	CBG is not in the fund, ignore and move to the next row

SELL NESF
	NESF is not in the fund, ignore and move to the next row

SELL MYI
	MYI is not in the fund, ignore and move to the next row

SELL MSLH
	MSLH is not in the fund, ignore and move to the next row

SELL REL
	REL is not in the fund, ignore and move to the next row

SELL BGSC
	BGSC is not in the fund, ignore and move to the next row

SELL FCSS
	FCSS is not in the fund, ignore and move to the next row

SELL RB
	RB is not in the fund, ignore and move to the next row

SELL DOM
	DOM is not in the fund, ignore and move to the next row

SELL NESF
	NESF is not in the fund, ignore and move to the next row

SELL CBG
	CBG is not in the fund, ignore and move to the next row

SELL UTG
	UTG is not in the fund, ignore and move to the next row

SELL GRG
	GRG is not in the fund, ignore and move to the next row

SELL MYI
	MYI is not in the fund, ignore and move t


SELL IMB
	IMB is not in the fund, ignore and move to the next row

SELL ABF
	ABF is not in the fund, ignore and move to the next row

SELL UTG
	UTG is not in the fund, ignore and move to the next row

SELL BDEV
	BDEV is not in the fund, ignore and move to the next row

SELL HILS
	HILS is not in the fund, ignore and move to the next row

SELL LGEN
	LGEN is not in the fund, ignore and move to the next row

SELL MONY
	MONY is not in the fund, ignore and move to the next row

SELL FEV
	FEV is not in the fund, ignore and move to the next row

SELL VCT
	VCT is not in the fund, ignore and move to the next row

SELL BRBY
	BRBY is not in the fund, ignore and move to the next row

SELL SMWH
	SMWH is not in the fund, ignore and move to the next row

SELL PSN
	PSN is not in the fund, ignore and move to the next row

SELL GRG
	GRG is not in the fund, ignore and move to the next row

SELL GFTU
	LEDGER ENTRY -> {'trade_type': 'sell', 'signal_prob': 0.5221210084058014, 'ticker': 'GFTU', 'trade_date':

...

SELL JE
	JE is not in the fund, ignore and move to the next row

SELL RDW
	RDW is not in the fund, ignore and move to the next row

SELL FDM
	FDM is not in the fund, ignore and move to the next row

SELL CCH
	CCH is not in the fund, ignore and move to the next row

SELL ASHM
	ASHM is not in the fund, ignore and move to the next row

SELL RDW
	RDW is not in the fund, ignore and move to the next row

SELL CCH
	CCH is not in the fund, ignore and move to the next row

SELL TRIG
	TRIG is not in the fund, ignore and move to the next row

SELL HGT
	HGT is not in the fund, ignore and move to the next row

SELL TW
	TW is not in the fund, ignore and move to the next row

SELL MYI
	MYI is not in the fund, ignore and move to the next row

SELL FXPO
	FXPO is not in the fund, ignore and move to the next row

SELL BVIC
	BVIC is not in the fund, ignore and move to the next row

SELL UTG
	UTG is not in the fund, ignore and move to the next row

SELL AAL
	AAL is not in the fund, ignore and move to the n

	UKW is not in the fund, ignore and move to the next row

SELL FUTR
	FUTR is not in the fund, ignore and move to the next row

SELL ADM
	ADM is not in the fund, ignore and move to the next row

SELL BAG
	BAG is not in the fund, ignore and move to the next row

SELL TMPL
	TMPL is not in the fund, ignore and move to the next row

SELL MSLH
	MSLH is not in the fund, ignore and move to the next row

SELL 3IN
	3IN is not in the fund, ignore and move to the next row

SELL AJB
	AJB is not in the fund, ignore and move to the next row

SELL AJB
	AJB is not in the fund, ignore and move to the next row

SELL SAFE
	SAFE is not in the fund, ignore and move to the next row

SELL DGE
	DGE is not in the fund, ignore and move to the next row

SELL SN
	SN is not in the fund, ignore and move to the next row

SELL GRI
	GRI is not in the fund, ignore and move to the next row

SELL ULVR
	ULVR is not in the fund, ignore and move to the next row

SELL FCSS
	FCSS is not in the fund, ignore and move to the next


SELL SPX
	SPX is not in the fund, ignore and move to the next row

SELL SPT
	SPT is not in the fund, ignore and move to the next row

SELL SSON
	SSON is not in the fund, ignore and move to the next row

SELL SDP
	SDP is not in the fund, ignore and move to the next row

SELL CLDN
	CLDN is not in the fund, ignore and move to the next row

SELL TEM
	TEM is not in the fund, ignore and move to the next row

SELL MSLH
	MSLH is not in the fund, ignore and move to the next row

SELL GFTU
	GFTU is not in the fund, ignore and move to the next row

SELL PRTC
	PRTC is not in the fund, ignore and move to the next row

SELL RAT
	RAT is not in the fund, ignore and move to the next row

SELL PAG
	PAG is not in the fund, ignore and move to the next row

SELL HL
	HL is not in the fund, ignore and move to the next row

SELL LMP
	LMP is not in the fund, ignore and move to the next row

SELL PHP
	PHP is not in the fund, ignore and move to the next row

SELL TBCG
	TBCG is not in the fund, ignore and move t


SELL RDW
	RDW is not in the fund, ignore and move to the next row

SELL ASHM
	ASHM is not in the fund, ignore and move to the next row

SELL PHP
	PHP is not in the fund, ignore and move to the next row

SELL GRG
	GRG is not in the fund, ignore and move to the next row

SELL ICP
	ICP is not in the fund, ignore and move to the next row

SELL LMP
	LMP is not in the fund, ignore and move to the next row

SELL ATST
	ATST is not in the fund, ignore and move to the next row

SELL AZN
	AZN is not in the fund, ignore and move to the next row

SELL RDW
	RDW is not in the fund, ignore and move to the next row

SELL AVV
	AVV is not in the fund, ignore and move to the next row

SELL LSE
	LSE is not in the fund, ignore and move to the next row

SELL PLP
	PLP is not in the fund, ignore and move to the next row

SELL CLDN
	CLDN is not in the fund, ignore and move to the next row

SELL RIO
	RIO is not in the fund, ignore and move to the next row

SELL SGRO
	SGRO is not in the fund, ignore and move to 

	GSK is not in the fund, ignore and move to the next row

SELL BGSC
	BGSC is not in the fund, ignore and move to the next row

SELL QQ
	QQ is not in the fund, ignore and move to the next row

SELL AVST
	AVST is not in the fund, ignore and move to the next row

SELL WIZZ
	WIZZ is not in the fund, ignore and move to the next row

SELL PLP
	PLP is not in the fund, ignore and move to the next row

SELL BRW
	BRW is not in the fund, ignore and move to the next row

SELL FERG
	FERG is not in the fund, ignore and move to the next row

SELL RMV
	RMV is not in the fund, ignore and move to the next row

SELL SGRO
	SGRO is not in the fund, ignore and move to the next row

SELL PHP
	PHP is not in the fund, ignore and move to the next row

SELL CLDN
	CLDN is not in the fund, ignore and move to the next row

SELL BYG
	BYG is not in the fund, ignore and move to the next row

SELL LMP
	LMP is not in the fund, ignore and move to the next row

SELL JFJ
	JFJ is not in the fund, ignore and move to the next

In [14]:
#Show summary
print('lgb_fund.st_val:£{:,.2f}'.format(lgb_fund.st_val/100))
print('lgb_fund.available:£{:,.2f}'.format(lgb_fund.available/100))
print('lgb_fund.codb:£{:,.2f}'.format(lgb_fund.codb/100))
print('lgb_fund.invested_value:£{:,.2f}'.format(lgb_fund.invested_value/100))
lgb_ledger_df = pd.DataFrame(lgb_fund.ledger,columns=[
            'trade_type'
            ,'signal_prob'
            ,'ticker'
            ,'trade_date'
            ,'spread'
            ,'price'
            ,'ask_price'
            ,'bid_price'
            ,'share_vol'
            ,'trade_value'
            ,'stamp_duty'
            ,'trade_cost'
            ,'spread_cost'
            ,'holding_value'
            ,'ledger_value'
            ,'invested_pre_trade'
            ,'invested_post_trade'
            ,'available_pre_trade'
            ,'available_post_trade'])
print('TRADE COUNT:{:,}'.format(len(lgb_ledger_df)))
lgb_ledger_df.head(50)

lgb_fund.st_val:£10,000.00
lgb_fund.available:£221.92
lgb_fund.codb:£1,535.19
lgb_fund.invested_value:£11,338.69
TRADE COUNT:98


,trade_type,signal_prob,ticker,trade_date,spread,price,ask_price,bid_price,share_vol,trade_value,stamp_duty,trade_cost,spread_cost,holding_value,ledger_value,invested_pre_trade,invested_post_trade,available_pre_trade,available_post_trade
0,buy,0.635030,AGR,2014-01-06,0.01,40.00,40.40,39.60,2456,99222.40,496.11,250,982.40,98240.00,-99968.51,0.00,98240.00,1000000.00,900031.49
1,buy,0.578365,DC,2014-01-06,0.01,277.00,279.77,274.23,354,99038.58,495.19,250,980.58,98058.00,-99783.77,98240.00,196298.00,900031.49,800247.72
2,buy,0.587106,EVR,2014-01-27,0.01,85.00,85.85,84.15,1156,99242.60,496.21,250,982.60,98260.00,-99988.81,196298.00,294558.00,800247.72,700258.91
3,buy,0.437862,888,2014-02-03,0.01,139.90,141.30,138.50,702,99192.60,495.96,250,982.10,98209.80,-99938.56,289363.60,387573.40,700258.91,600320.35
4,buy,0.427413,GRG,2014-02-03,0.01,508.00,513.08,502.92,193,99024.44,495.12,250,980.44,98044.00,-99769.56,387573.40,485617.40,600320.35,500550.79
5,buy,0.371587,CPG,2014-02-03,0.01,988.42,998.30,978.54,99,98831.70,494.16,250,978.54,97853.58,-99575.86,485617.40,583470.98,500550.79,400974.93
6,buy,0.485255,BA,2014-02-17,0.01,412.40,416.52,408.28,238,99131.76,495.66,250,981.51,98151.20,-99877.42,612775.66,710926.86,400974.93,301097.51
7,buy,0.472421,HL,2014-02-17,0.01,1317.00,1330.17,1303.83,74,98432.58,492.16,250,974.58,97458.00,-99174.74,710926.86,808384.86,301097.51,201922.77
8,buy,0.705887,CCH,2014-02-24,0.01,1452.00,1466.52,1437.48,67,98256.84,491.28,250,972.84,97284.00,-98998.12,808384.86,905668.86,201922.77,102924.65
9,buy,0.415681,WPP,2014-03-03,0.01,1257.00,1269.57,1244.43,78,99026.46,495.13,250,980.46,98046.00,-99771.59,918245.40,1016291.40,102924.65,3153.06


In [15]:
#Get the completed trades
lgb_trades_df = completed_trades(lgb_ledger_df,lgb_fund.st_val,lgb_fund.cur_holdings)
lgb_trades_df

TRADE COUNT -> 42
TOTAL PROFIT/LOSS -> £4,191.08

OPEN TRADES -> 14
OPEN TRADES VALUE -> £12,869.74

TOTAL ROI VALUE -> £17,060.82
TOTAL ROI % -> 70.61%

ROI STATS:
        AVERAGE ROI % -> 6.55
        AVERAGE POS ROI % -> 27.82
        POS COUNT -> 28
        AVERAGE NEG ROI % -> -14.73
        NEG COUNT -> 28
    
PERIODS HELD STATS:
        AVERAGE PERIODS HELD -> 50.71
        AVERAGE POS ROI PERIODS HELD -> 64.43
        AVERAGE NEG ROI PERIODS HELD -> 34.11
        


,ticker,share_vol,buy_price,buy_spend,buy_prob,buy_date,sell_price,sell_spend,sell_prob,sell_date,periods_held,profit_loss,holding_value,roi
0,AGR,2456,40.00,99968.51,0.635030,2014-01-06,45.50,110392.80,0.417884,2014-08-11,31.0,10424.29,111748.00,0.117832
1,AGR,1600,61.40,99962.08,0.401890,2017-10-30,56.80,89718.00,0.427979,2018-08-13,41.0,-10244.08,90880.00,-0.090855
2,DC,354,277.00,99783.77,0.578365,2014-01-06,464.30,162469.64,0.518243,2014-12-22,50.0,62685.87,164362.20,0.647184
3,EVR,1156,85.00,99988.81,0.587106,2014-01-27,296.00,338504.24,0.504505,2017-08-21,186.0,238515.43,342176.00,2.422143
4,888,702,139.90,99938.56,0.437862,2014-02-03,141.25,97917.68,0.499549,2014-12-08,44.0,-2020.88,99157.50,-0.007815
5,GRG,193,508.00,99769.56,0.427413,2014-02-03,547.50,104359.86,0.541788,2014-04-28,12.0,4590.30,105667.50,0.059116
6,CPG,99,988.42,99575.86,0.371587,2014-02-03,1154.72,112923.83,0.575948,2014-06-30,21.0,13347.97,114317.28,0.148042
7,BA,238,412.40,99877.42,0.485255,2014-02-17,414.90,97508.50,0.457035,2014-05-12,12.0,-2368.92,98746.20,-0.011326
8,HL,74,1317.00,99174.74,0.472421,2014-02-17,1130.00,82533.80,0.674585,2015-02-23,53.0,-16640.94,83620.00,-0.156842
9,CCH,67,1452.00,98998.12,0.705887,2014-02-24,1786.00,118215.38,0.445748,2017-01-30,153.0,19217.26,119662.00,0.208730


In [16]:
#Export
lgb_ledger_df.to_csv(path+r'\fund_ledger_lgb.csv')
lgb_trades_df.to_csv(path+r'\fund_trades_lgb.csv')